## Quantitative Portfolio Analysis

### Objective  
To evaluate and optimize a portfolio of U.S. stocks using financial theories such as the Capital Asset Pricing Model (CAPM) and Mean-Variance Optimization. The goal is to understand each asset’s contribution to risk and return, assess performance using factor models, and construct efficient portfolios.

---

### Project Summary  
This project applies quantitative techniques in portfolio analysis to:
- Fetch historical monthly price data for 10 selected stocks and the S&P 500 ETF (SPY).
- Calculate monthly log returns, covariance matrices, and statistical metrics.
- Use CAPM to estimate each asset's sensitivity to market returns (beta) and excess return (alpha).
- Extend analysis with multi-factor models including size, value, and momentum factors.
- Construct active and passive portfolios using the Single Index Model.
- Compare expected performance using the Sharpe Ratio and optimize portfolio weights.

---

### Methodology Used

- **Data Collection**  
  Retrieved 5 years of monthly stock prices via `yfinance`.

- **Return & Risk Estimation**  
  - Computed log returns and variance-covariance matrix  
  - Estimated Beta and Alpha using CAPM

- **Statistical Testing**  
  - Performed regression analysis to assess alpha significance  
  - Compared calculated Betas with Yahoo Finance data

- **Multi-Factor Modeling**  
  - Included Fama-French 3-Factor (SMB, HML) and Momentum (MOM)

- **Portfolio Optimization**  
  - Built Single Index Model (SIM) portfolios  
  - Derived active and passive weights  
  - Compared Sharpe Ratios

---

### Key Insights

- Most stock alphas were not statistically significant, validating CAPM assumptions.
- NVIDIA (NVDA) was the only asset with statistically significant alpha.
- Multi-factor models altered alpha values but did not enhance significance across the board.
- Optimized portfolios showed better Sharpe Ratios than SPY, validating the benefit of diversification and active allocation.


In [24]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime

### Data Collection using yfinance

This code block performs the following tasks:

1. **Defines a list of stock tickers** including major U.S. companies and the SPY ETF (used as a market index).
2. **Initializes an empty DataFrame** called `prices` to store the historical price data.
3. **Sets the date range** to fetch 5 years of monthly data (from September 30, 2019, to September 30, 2024).
4. **Loops through each ticker** and downloads monthly adjusted closing prices using the `yfinance` library.
5. **Concatenates each stock's data** into the `prices` DataFrame, with each column named after its corresponding ticker.

In [25]:
tickers = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA', 'NVDA', 'META', 'AMD', 'PG', 'V', 'SPY']
prices = pd.DataFrame()
end_date = '2024-09-30'  
start_date = '2019-09-30'

In [26]:
for ticker in tickers:
    data = yf.download(ticker, start=start_date, end=end_date, interval="1mo")['Adj Close']
    data.name = ticker
    prices = pd.concat([prices, data], axis=1)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


### Log Return Calculation

This section computes **monthly logarithmic returns**, which are a standard way of expressing investment performance over time. Here's why they are used:

1. **Time-Additive Property**:  
   Log returns are additive across time periods — the total return over multiple periods can be obtained by summing individual log returns. This is particularly useful in portfolio theory and risk modeling.

2. **Better Statistical Properties**:  
   Log returns tend to be more normally distributed than simple returns, especially over shorter intervals. This makes them a better fit for linear models like CAPM and for assumptions underlying many financial theories.

3. **Relative and Scale-Invariant**:  
   Log returns represent percentage changes in a way that is independent of the asset's price scale. This makes comparisons across assets more consistent.

4. **Accurate for Compounding**:  
   For small returns, log returns closely approximate percentage returns. Over multiple periods, they compound correctly, especially when working with continuous processes like Brownian motion used in stochastic models.

By transforming raw price data into log returns, we ensure the data is more suitable for regression analysis, correlation and covariance calculation, and portfolio optimization techniques.


In [27]:
logreturns = np.log(prices / prices.shift(1))
logreturns = logreturns.dropna()
logreturns.head(10)

Ticker,AAPL,MSFT,GOOGL,AMZN,TSLA,NVDA,META,AMD,PG,V,SPY
Date,,,,,,,,,,,
2019-11-01,0.071696,0.054365,0.035346,0.013496,0.046592,0.075294,0.050813,0.143101,-0.013506,0.031101,0.035558
2019-12-01,0.097202,0.044298,0.026709,0.025786,0.237360,0.082901,0.017745,0.158193,0.023001,0.019880,0.023737
2020-01-01,0.052602,0.076456,0.067402,0.083480,0.441578,0.004791,-0.016407,0.024554,-0.002244,0.057244,0.004500
2020-02-01,-0.124201,-0.049492,-0.067507,-0.064233,0.026424,0.133029,-0.047882,-0.032875,-0.089924,-0.090467,-0.082475
2020-03-01,-0.069944,-0.024173,-0.142010,0.034421,-0.242781,-0.023650,-0.143145,0.000000,-0.028941,-0.119090,-0.139247
2020-04-01,0.144424,0.127800,0.147558,0.238150,0.400210,0.103279,0.204799,0.141443,0.069102,0.103671,0.125408
2020-05-01,0.078964,0.022293,0.062476,-0.012867,0.065730,0.194462,0.094906,0.026558,-0.010035,0.088409,0.046545
2020-06-01,0.140190,0.107645,-0.010850,0.121834,0.257109,0.067760,0.008758,-0.022367,0.031002,-0.008986,0.013189
2020-07-01,0.152834,0.007343,0.048117,0.137249,0.281421,0.111646,0.110776,0.386468,0.092211,-0.014444,0.061614


### Variance-Covariance Matrix Calculation

The code calculates the **variance-covariance matrix** of asset returns to quantify the relationship between different assets' risks and how they move together. This matrix is a crucial input for portfolio risk assessment and optimization, helping to determine how combining assets affects overall portfolio volatility.


In [ ]:
varcov_matrix = logreturns.cov()
varcov_matrix

Ticker,AAPL,MSFT,GOOGL,AMZN,TSLA,NVDA,META,AMD,PG,V,SPY
Ticker,,,,,,,,,,,
AAPL,0.006832,0.003660,0.003520,0.005257,0.011574,0.006305,0.003983,0.006793,0.001454,0.003123,0.003371
MSFT,0.003660,0.003986,0.003303,0.004072,0.006356,0.005944,0.004247,0.005864,0.001085,0.001875,0.002446
GOOGL,0.003520,0.003303,0.006002,0.004637,0.007364,0.006479,0.004208,0.006210,0.000534,0.002290,0.003010
AMZN,0.005257,0.004072,0.004637,0.008586,0.011788,0.008633,0.005256,0.008668,0.000228,0.002234,0.003136
TSLA,0.011574,0.006356,0.007364,0.011788,0.040185,0.012136,0.007587,0.013317,-0.000433,0.003962,0.005774
NVDA,0.006305,0.005944,0.006479,0.008633,0.012136,0.019797,0.008603,0.015281,0.000532,0.002944,0.004702
META,0.003983,0.004247,0.004208,0.005256,0.007587,0.008603,0.015188,0.008655,0.001866,0.002515,0.003332
AMD,0.006793,0.005864,0.006210,0.008668,0.013317,0.015281,0.008655,0.021668,0.001226,0.002871,0.004728
PG,0.001454,0.001085,0.000534,0.000228,-0.000433,0.000532,0.001866,0.001226,0.002425,0.001437,0.001136


### Beta Calculation Using Covariance Matrix

This code calculates the beta of each stock relative to the market index (SPY), which indicates how sensitive each stock is to overall market movements.

In [29]:
market_variance = varcov_matrix.loc['SPY', 'SPY']
beta = varcov_matrix['SPY'] / market_variance
beta

Ticker
AAPL     1.183272
MSFT     0.858615
GOOGL    1.056544
AMZN     1.100565
TSLA     2.026598
NVDA     1.650366
META     1.169394
AMD      1.659430
PG       0.398707
V        0.928940
SPY      1.000000
Name: SPY, dtype: float64

### Alpha Calculation Using CAPM

This code calculates the alpha for each stock based on the Capital Asset Pricing Model (CAPM), representing the stock's excess return beyond what is explained by market movements.

In [30]:
r_f = 0.042
avg_stockreturns = logreturns.mean()
avg_marketreturns = logreturns['SPY'].mean()
expected_returns = r_f + beta * (avg_marketreturns - r_f)
alpha = avg_stockreturns - expected_returns
alpha 

Ticker
AAPL     1.637186e-02
MSFT     3.112720e-03
GOOGL    6.074451e-03
AMZN     3.506213e-03
TSLA     6.143337e-02
NVDA     6.147404e-02
META     1.159286e-02
AMD      3.439616e-02
PG      -2.236802e-02
V       -6.304800e-03
SPY      1.734723e-18
dtype: float64

### CAPM Regression and Alpha Significance Testing

This block performs a linear regression of excess stock returns on excess market returns (SPY) to estimate each stock’s alpha and beta. It also tests if the alpha is statistically significant using the t-test and p-value.

In [31]:
import statsmodels.api as sm

results = pd.DataFrame(columns=['Alpha', 'Beta', 'T-Stat', 'P-Value', 'Significant'])  
for ticker in [t for t in tickers if t != 'SPY']:
    excess_stock_returns = logreturns[ticker] - (0.042 / 12)
    excess_market_returns = logreturns['SPY'] - (0.042 / 12)
    X = sm.add_constant(excess_market_returns)
    y = excess_stock_returns
    model = sm.OLS(y, X).fit()

    alpha_value = model.params['const']
    beta_value = model.params['SPY']
    alpha_tstat = model.tvalues['const']
    alpha_pvalue = model.pvalues['const']
    significant = alpha_pvalue < 0.05
    
    results.loc[ticker] = [alpha_value, beta_value, alpha_tstat, alpha_pvalue, significant]

results


,Alpha,Beta,T-Stat,P-Value,Significant
AAPL,0.009316,1.183272,1.313418,0.194305,False
MSFT,0.008556,0.858615,1.481267,0.144043,False
GOOGL,0.003898,1.056544,0.551527,0.583428,False
AMZN,-0.000366,1.100565,-0.038346,0.969546,False
TSLA,0.021909,2.026598,0.975847,0.333264,False
NVDA,0.036435,1.650366,2.496387,0.015462,True
META,0.005071,1.169394,0.358741,0.721114,False
AMD,0.009008,1.659430,0.575947,0.566918,False
PG,0.000782,0.398707,0.132348,0.895175,False
V,-0.003569,0.928940,-0.591006,0.556853,False


In [32]:
file = 'Fame French Factors w MoM.CSV'
ff_data = pd.read_csv(file)
ff_data.head(10)

,Date,MoM,SMB,HML
0,201911,-2.64,0.45,-1.99
1,201912,-1.87,0.97,1.78
2,202001,5.97,-4.40,-6.25
3,202002,-0.35,0.04,-3.80
4,202003,7.96,-8.24,-13.88
5,202004,-5.26,2.56,-1.34
6,202005,0.41,1.99,-4.85
7,202006,-0.73,1.97,-2.23
8,202007,7.59,-3.18,-1.44
9,202008,0.44,-0.95,-2.88


In [33]:
ff_data['Date'] = pd.to_datetime(ff_data['Date'], format='%Y%m')
merged_df = pd.merge(logreturns, ff_data, on='Date', how='inner')
merged_df.head(10)

,Date,AAPL,MSFT,GOOGL,AMZN,TSLA,NVDA,META,AMD,PG,V,SPY,MoM,SMB,HML
0,2019-11-01,0.071696,0.054365,0.035346,0.013496,0.046592,0.075294,0.050813,0.143101,-0.013506,0.031101,0.035558,-2.64,0.45,-1.99
1,2019-12-01,0.097202,0.044298,0.026709,0.025786,0.237360,0.082901,0.017745,0.158193,0.023001,0.019880,0.023737,-1.87,0.97,1.78
2,2020-01-01,0.052602,0.076456,0.067402,0.083480,0.441578,0.004791,-0.016407,0.024554,-0.002244,0.057244,0.004500,5.97,-4.40,-6.25
3,2020-02-01,-0.124201,-0.049492,-0.067507,-0.064233,0.026424,0.133029,-0.047882,-0.032875,-0.089924,-0.090467,-0.082475,-0.35,0.04,-3.80
4,2020-03-01,-0.069944,-0.024173,-0.142010,0.034421,-0.242781,-0.023650,-0.143145,0.000000,-0.028941,-0.119090,-0.139247,7.96,-8.24,-13.88
5,2020-04-01,0.144424,0.127800,0.147558,0.238150,0.400210,0.103279,0.204799,0.141443,0.069102,0.103671,0.125408,-5.26,2.56,-1.34
6,2020-05-01,0.078964,0.022293,0.062476,-0.012867,0.065730,0.194462,0.094906,0.026558,-0.010035,0.088409,0.046545,0.41,1.99,-4.85
7,2020-06-01,0.140190,0.107645,-0.010850,0.121834,0.257109,0.067760,0.008758,-0.022367,0.031002,-0.008986,0.013189,-0.73,1.97,-2.23
8,2020-07-01,0.152834,0.007343,0.048117,0.137249,0.281421,0.111646,0.110776,0.386468,0.092211,-0.014444,0.061614,7.59,-3.18,-1.44
9,2020-08-01,0.194234,0.095394,0.090892,0.086601,0.554719,0.231106,0.144821,0.159505,0.059820,0.107412,0.067469,0.44,-0.95,-2.88


### Fama-French 3-Factor and Momentum Model Regression

This block extends the CAPM by including additional explanatory factors — SMB (size), HML (value), and MOM (momentum) — to better capture asset returns and test the adjusted alpha's statistical significance.

In [34]:
results_ff = pd.DataFrame(columns=['Alpha', 'Beta_MKT', 'Beta_SMB', 'Beta_HML', 'Beta_MOM', 
                                   'T-Stat', 'P-Value', 'Significant'])

for ticker in [t for t in tickers if t != 'SPY']:
    excess_stock_returns = merged_df[ticker] - (0.042 / 12)
    X = merged_df[['SPY', 'SMB', 'HML', 'MoM']]
    X = sm.add_constant(X) 
    y = excess_stock_returns

    model = sm.OLS(y, X).fit()

    alpha = model.params['const']
    beta_mkt = model.params['SPY']
    beta_smb = model.params['SMB']
    beta_hml = model.params['HML']
    beta_mom = model.params['MoM']
    alpha_tstat = model.tvalues['const']
    alpha_pvalue = model.pvalues['const']
    significant = alpha_pvalue < 0.05

    results_ff.loc[ticker] = [alpha, beta_mkt, beta_smb, beta_hml, beta_mom, 
                                    alpha_tstat, alpha_pvalue, significant]

results_ff
    

,Alpha,Beta_MKT,Beta_SMB,Beta_HML,Beta_MOM,T-Stat,P-Value,Significant
AAPL,0.004929,1.211445,0.000089,-0.005186,0.000063,0.745568,0.459162,False
MSFT,0.004197,0.935780,-0.003799,-0.004673,-0.000082,0.938006,0.352418,False
GOOGL,0.001399,0.952728,-0.003186,-0.002762,-0.005061,0.207813,0.836157,False
AMZN,-0.003724,1.080977,-0.000474,-0.009823,-0.002409,-0.485801,0.629074,False
TSLA,0.019682,1.828964,0.015439,-0.017335,-0.002002,0.948240,0.347232,False
NVDA,0.029664,1.734346,-0.001353,-0.011963,0.000035,2.289047,0.026013,True
META,0.001779,1.049572,-0.009913,-0.005732,-0.009036,0.134648,0.893391,False
AMD,0.001915,1.745308,-0.003177,-0.009315,-0.000291,0.128339,0.898358,False
PG,-0.003040,0.513354,-0.007132,0.002521,0.000751,-0.579857,0.564422,False
V,-0.006462,0.869288,-0.004195,0.001471,-0.003466,-1.078219,0.285729,False


### Portfolio Construction Using Single Index Model (SIM)

This block constructs active, passive, and overall portfolio weights using the Single Index Model. It combines alpha-based active weights (adjusted for tracking error) and beta-based passive weights to form a blended investment strategy.


In [35]:
tracking_errors = {}
for ticker in [t for t in tickers if t != 'SPY']:
    excess_stock_returns = logreturns[ticker] - (0.042 / 12)
    excess_market_returns = logreturns['SPY'] - (0.042 / 12)
    X = sm.add_constant(excess_market_returns)
    y = excess_stock_returns
    model = sm.OLS(y, X).fit()

    residuals = model.resid
    tracking_errors[ticker] = np.std(residuals)

active_weights = {}
alpha_te_sum = sum(results.loc[ticker, 'Alpha'] / tracking_errors[ticker] for ticker in results.index)

for ticker in results.index:
    active_weights[ticker] = (results.loc[ticker, 'Alpha'] / tracking_errors[ticker]) / alpha_te_sum

market_variance = logreturns['SPY'].var()  
betas = {ticker: varcov_matrix.loc[ticker, 'SPY'] / market_variance for ticker in results.index}
beta_sum = sum(betas.values())

passive_weights = {ticker: betas[ticker] / beta_sum for ticker in betas.keys()}

overall_weights = {ticker: active_weights[ticker] + passive_weights[ticker] for ticker in results.index}

print('Active Portfolio Weights:')
print(pd.Series(active_weights))

print('Passive Portfolio Weights:')
print(pd.Series(passive_weights))

print('Overall Portfolio Weights:')
print(pd.Series(overall_weights))

Active Portfolio Weights:
AAPL     0.181008
MSFT     0.204140
GOOGL    0.076008
AMZN    -0.005285
TSLA     0.134486
NVDA     0.344038
META     0.049440
AMD      0.079374
PG       0.018239
V       -0.081449
dtype: float64
Passive Portfolio Weights:
AAPL     0.098340
MSFT     0.071358
GOOGL    0.087808
AMZN     0.091467
TSLA     0.168428
NVDA     0.137160
META     0.097187
AMD      0.137913
PG       0.033136
V        0.077203
dtype: float64
Overall Portfolio Weights:
AAPL     0.279348
MSFT     0.275498
GOOGL    0.163816
AMZN     0.086182
TSLA     0.302914
NVDA     0.481198
META     0.146627
AMD      0.217287
PG       0.051375
V       -0.004246
dtype: float64


### Sharpe Ratio Comparison: SIM Portfolio vs. Market (SPY)

This block calculates and compares the Sharpe Ratio of the constructed Single Index Model (SIM) portfolio against the benchmark market index (SPY), to assess which offers better risk-adjusted returns.

In [36]:
r_f = 0.042 / 12

expected_stock_returns = [r_f + beta[ticker] * (logreturns['SPY'].mean() - r_f) for ticker in tickers if ticker != 'SPY']
expected_stock_returns = np.array(expected_stock_returns)

weights = np.array([overall_weights[ticker] for ticker in tickers if ticker != 'SPY'])
expected_SIM_return = np.dot(weights, expected_stock_returns)

cov_matrix = logreturns.loc[:, tickers[:-1]].cov().to_numpy()
portfolio_variance = np.dot(weights.T, np.dot(cov_matrix, weights))
portfolio_volatility = np.sqrt(portfolio_variance)

sharpe_ratio_SIM = (expected_SIM_return - r_f) / portfolio_volatility

expected_SPY_return = logreturns['SPY'].mean()
spy_volatility = logreturns['SPY'].std()
sharpe_ratio_SPY = (expected_SPY_return - r_f) / spy_volatility

print("SIM SR:", sharpe_ratio_SIM)
print("Market SR:", sharpe_ratio_SPY)

SIM SR: 0.1440891253917534
Market SR: 0.16036422987073368


# Part 2

### Mean-Variance Portfolio Optimization with Constraints

This block solves for the minimum variance portfolio using selected stocks, subject to constraints (weights sum to 1 and each asset has at least 5% allocation), using the `scipy.optimize.minimize` function.


In [37]:
import numpy as np
import pandas as pd
import yfinance as yf
from scipy.optimize import minimize

tickers = ['META', 'PG', 'AMD', 'V']
prices = pd.DataFrame()

for ticker in tickers:
    data = yf.download(ticker, period="5y", interval="1mo")['Adj Close']
    data.name = ticker
    prices = pd.concat([prices, data], axis=1)

returns = prices.pct_change().dropna()

cov_matrix = returns.cov().values
print(cov_matrix)

n = len(tickers)

constraints = [
    {'type': 'eq', 'fun': lambda w: np.sum(w) - 1}, 
    {'type': 'ineq', 'fun': lambda w: w - 0.05}  
]

initial_weights = np.ones(n) / n

def portfolio_variance(w, cov_matrix):
    return w.T @ cov_matrix @ w

result = minimize(
    portfolio_variance,
    initial_weights,
    args=(cov_matrix,),
    constraints=constraints,
    bounds=[(0.05, 1)] * n,  
    method='SLSQP'
)

optimal_weights = result.x

print("Optimal Portfolio Weights:")
for i, ticker in enumerate(tickers):
    print(f"  {ticker}: {optimal_weights[i]:.4f}")

optimal_variance = portfolio_variance(optimal_weights, cov_matrix)
print(f"Minimum Portfolio Variance: {optimal_variance:.6f}")


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

[[0.01438195 0.0019426  0.00893488 0.00255822]
 [0.0019426  0.0026884  0.00145188 0.00153524]
 [0.00893488 0.00145188 0.02366394 0.00249915]
 [0.00255822 0.00153524 0.00249915 0.00471674]]
Optimal Portfolio Weights:
  META: 0.0500
  PG: 0.6798
  AMD: 0.0500
  V: 0.2202
Minimum Portfolio Variance: 0.002413


### Tangency Portfolio Optimization (Maximum Sharpe Ratio)

This code computes the optimal portfolio on the Capital Market Line (CML) by maximizing the Sharpe Ratio. It determines asset weights that offer the best risk-adjusted return relative to the risk-free rate.


In [38]:
expected_returns = returns.mean().values
risk_free_rate = 0.042 / 12

def negative_sharpe_ratio(weights, expected_returns, cov_matrix, risk_free_rate):
    portfolio_return = np.dot(weights, expected_returns)
    portfolio_variance = np.dot(weights.T, np.dot(cov_matrix, weights))
    portfolio_std_dev = np.sqrt(portfolio_variance)
    sharpe_ratio = (portfolio_return - risk_free_rate) / portfolio_std_dev
    return -sharpe_ratio

constraints = [{'type': 'eq', 'fun': lambda w: np.sum(w) - 1}]
bounds = [(0, 1) for _ in range(len(tickers))]
initial_weights = np.ones(len(tickers)) / len(tickers)

result = minimize(
    negative_sharpe_ratio,
    initial_weights,
    args=(expected_returns, cov_matrix, risk_free_rate),
    method='SLSQP',
    bounds=bounds,
    constraints=constraints
)

optimal_weights = result.x
tangent_portfolio_return = np.dot(optimal_weights, expected_returns)
tangent_portfolio_variance = np.dot(optimal_weights.T, np.dot(cov_matrix, optimal_weights))
tangent_portfolio_std_dev = np.sqrt(tangent_portfolio_variance)
sharpe_ratio_tangent = (tangent_portfolio_return - risk_free_rate) / tangent_portfolio_std_dev

print("Tangent Portfolio Weights:")
for i, ticker in enumerate(tickers):
    print(f"  {ticker}: {optimal_weights[i]:.4f}")

print(f"Tangent Portfolio Return: {tangent_portfolio_return:.6f}")
print(f"Tangent Portfolio Volatility: {tangent_portfolio_std_dev:.6f}")
print(f"Tangent Portfolio Sharpe Ratio: {sharpe_ratio_tangent:.6f}")


Tangent Portfolio Weights:
  META: 0.4199
  PG: 0.2218
  AMD: 0.2019
  V: 0.1563
Tangent Portfolio Return: 0.020658
Tangent Portfolio Volatility: 0.079723
Tangent Portfolio Sharpe Ratio: 0.215225


### Tangent Portfolio Sharpe Ratio Calculation

This section calculates and prints the Sharpe Ratio of the tangent (optimal) portfolio using the optimized weights and the portfolio's covariance matrix.

In [43]:
weights_tangent = optimal_weights
cov_matrix_tangent = cov_matrix

portfolio_variance = np.dot(weights_tangent.T, np.dot(cov_matrix_tangent, weights_tangent))
portfolio_volatility = np.sqrt(portfolio_variance)

print("Tangent Portfolio Sharpe Ratio:", round(sharpe_ratio_tangent, 6))


Tangent Portfolio Sharpe Ratio: 0.215225


### Investment Allocation and Margin Requirement Calculation

This block computes the dollar investment per stock based on optimal weights and total capital, then calculates the required margin assuming a 50% initial margin rate.


In [40]:
investment_amount = 10000
initial_margin = 0.5

investment_per_stock = investment_amount * optimal_weights
margin_required = investment_per_stock * initial_margin

print("Investment Per Stock:")
for i, ticker in enumerate(tickers):
    print(ticker, round(investment_per_stock[i], 2))

print("Margin Required Per Stock:")
for i, ticker in enumerate(tickers):
    print(ticker, round(margin_required[i], 2))


Investment Per Stock:
META 4199.3
PG 2218.23
AMD 2019.19
V 1563.29
Margin Required Per Stock:
META 2099.65
PG 1109.11
AMD 1009.59
V 781.64


### Margin Requirement Calculation for Long and Short Positions

This block calculates the total margin required for each stock, accounting for both long and short positions. A 50% margin is applied to shorts, and 100% to longs, based on optimal portfolio weights.


In [50]:
short_positions = np.array([weight < 0 for weight in optimal_weights])
long_positions = ~short_positions

investment_per_stock = investment_amount * optimal_weights
margin_required_short = np.where(short_positions, investment_per_stock * 0.5, 0)
margin_required_long = np.where(long_positions, investment_per_stock, 0)

total_margin_required = margin_required_short + margin_required_long

print("Margin Required Per Stock:")
for i, ticker in enumerate(tickers):
    print(ticker, round(total_margin_required[i], 2))


Margin Required Per Stock:
META 4199.3
PG 2218.23
AMD 2019.19
V 1563.29


### Adjusted Portfolio Weights and Sharpe Ratio with 75% Margin

This block recalculates portfolio weights and the Sharpe Ratio by adjusting for leverage, assuming a 75% margin. It reflects how leverage impacts risk-adjusted performance.

In [47]:
investment_amount = 10000
initial_margin = 0.75

investment_per_stock = investment_amount * optimal_weights
margin_required = investment_per_stock * initial_margin

adjusted_investment = investment_amount / initial_margin
adjusted_weights = optimal_weights * adjusted_investment / investment_amount

adjusted_expected_return = np.dot(adjusted_weights, expected_returns)
adjusted_portfolio_variance = np.dot(adjusted_weights.T, np.dot(cov_matrix, adjusted_weights))
adjusted_portfolio_volatility = np.sqrt(adjusted_portfolio_variance)

sharpe_ratio_adjusted = (adjusted_expected_return - risk_free_rate) / adjusted_portfolio_volatility

print("Adjusted Portfolio Weights with 75% Margin:")
for i, ticker in enumerate(tickers):
    print(ticker, round(adjusted_weights[i], 4))

print("Adjusted Sharpe Ratio with 75% Margin:", round(sharpe_ratio_adjusted, 6))



Adjusted Portfolio Weights with 75% Margin:
META 0.5599
PG 0.2958
AMD 0.2692
V 0.2084
Adjusted Sharpe Ratio with 75% Margin: 0.226201
